# Receipt Data Extraction: Using AI_EXTRACT for Direct PDF Processing

## Overview
This notebook demonstrates how to use Snowflake's AI_EXTRACT function to directly extract structured information from ad-campaign receipt PDFs on a Snowflake stage. AI_EXTRACT can process PDF files directly without requiring a separate parsing step.

## What We'll Accomplish
- Extract structured receipt data directly from PDF files using `AI_EXTRACT` with the `file` parameter
- Transform PDF receipts into structured, analyzable data in a single step
- Create analytics views for reporting and insights

## Prerequisites
- Access to Snowflake with Cortex AI features enabled
- Receipts uploaded to `@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS` stage
- RECEIPTS_PROCESSING_DB database and RAW schema configured
- Appropriate permissions for the ETL service role


In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# Use Snowpark for our analyses
from snowflake.snowpark.context import get_active_session
session = get_active_session()


## Step 1: Environment Setup and Session Initialization

Setting up our environment by importing necessary packages and establishing a Snowflake session.

### Key Components:
- **Streamlit**: For building interactive applications
- **Pandas**: For data manipulation and analysis
- **Snowpark Session**: Connection to Snowflake and access to Cortex AI capabilities

The `get_active_session()` retrieves our Snowflake session for executing SQL and using AI features.

## Step 2: Set Database and Schema Context
Setting the working context for our session:

- **Database**: `RECEIPTS_PROCESSING_DB` - Our receipt processing database
- **Schema**: `RAW` - The schema containing our receipts stage

This ensures all operations execute within the correct context without needing to fully qualify object names.


In [ ]:
session.sql("USE ROLE SYSADMIN").collect()

# Set warehouse for AI_EXTRACT processing (separate from AI_COMPLETE for cost tracking)
session.sql("USE WAREHOUSE RECEIPTS_AI_EXTRACT_WH").collect()

session.sql("ALTER WAREHOUSE RECEIPTS_AI_EXTRACT_WH SET WAREHOUSE_SIZE='XSMALL'").collect()

session.use_database('RECEIPTS_PROCESSING_DB')
session.use_schema('RAW')


## Step 3: Import Processing Libraries

Importing libraries for document AI processing:

### Key Imports:
- **JSON**: For handling structured data extracted by AI_EXTRACT

Note: AI_EXTRACT requires minimal configuration compared to AI_COMPLETE, so we don't need Pydantic schemas or complex prompt management.


In [ ]:
#import json


## Step 4: Explore Available Receipts

Before processing, let's see what receipt files are available in our stage.

### What This Shows:
- File names and paths of receipts ready for processing
- File sizes and metadata
- Upload timestamps

The `DIRECTORY()` function provides a view of all files in the `@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS` stage, essential for understanding our data source.


In [ ]:
--REMOVE @RECEIPTS_PROCESSING_DB.RAW.RECEIPTS; -- REMOVES ALL FILES FROM THE STAGE
ALTER STAGE RECEIPTS_PROCESSING_DB.RAW.RECEIPTS REFRESH;
SELECT * FROM DIRECTORY(@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS);


## Step 5: Define Response Schema

AI_EXTRACT supports structured JSON schemas via the `responseFormat` parameter. This allows us to:
- Define nested object structures (vendor, transaction, customer, campaign, etc.)
- Specify field types and requirements  
- Add descriptions to guide extraction
- Ensure consistent output format

The `resp_schema` defines the complete structure for our receipt data extraction.

In [ ]:
# Complete JSON schema for AI_EXTRACT - all fields as type: 'array'
resp_schema = """
{
    'schema': {
        'type': 'object',
        'properties': {
            'vendor': {
                'description': 'Vendor or advertising service provider information',
                'type': 'object',
                'properties': {
                    'vendor_name': {
                        'description': 'Name of the vendor',
                        'type': 'array'
                    }
                }
            },
            'transaction': {
                'description': 'Transaction and receipt details',
                'type': 'object',
                'properties': {
                    'receipt_id': {
                        'description': 'Receipt or invoice ID number',
                        'type': 'array'
                    },
                    'date': {
                        'description': 'Date of the transaction',
                        'type': 'array'
                    },
                    'payment_method': {
                        'description': 'Payment method used',
                        'type': 'array'
                    }
                }
            },
            'customer': {
                'description': 'Customer or client information',
                'type': 'object',
                'properties': {
                    'customer_name': {
                        'description': 'Name of the customer contact',
                        'type': 'array'
                    },
                    'company_name': {
                        'description': 'Name of the customer company',
                        'type': 'array'
                    }
                }
            },
            'campaign': {
                'description': 'Advertising campaign details',
                'type': 'object',
                'properties': {
                    'name': {
                        'description': 'Name of the advertising campaign',
                        'type': 'array'
                    },
                    'content_types': {
                        'description': 'Types of content (Display, Video, etc.)',
                        'type': 'array'
                    },
                    'ad_formats': {
                        'description': 'Ad formats used in the campaign',
                        'type': 'array'
                    },
                    'period_startdate': {
                        'description': 'Campaign period start date',
                        'type': 'array'
                    },
                    'period_enddate': {
                        'description': 'Campaign period end date',
                        'type': 'array'
                    }
                }
            },
            'financials': {
                'description': 'Financial details from the receipt',
                'type': 'object',
                'properties': {
                    'line_items': {
                        'description': 'Individual line items on the receipt',
                        'type': 'array'
                    },
                    'subtotal': {
                        'description': 'Subtotal amount before tax',
                        'type': 'array'
                    },
                    'tax': {
                        'description': 'Tax amount',
                        'type': 'array'
                    },
                    'total': {
                        'description': 'Total amount charged',
                        'type': 'array'
                    }
                }
            },
            'metrics': {
                'description': 'Campaign performance metrics',
                'type': 'object',
                'properties': {
                    'cpm': {
                        'description': 'Cost per thousand impressions (CPM)',
                        'type': 'array'
                    },
                    'ctr': {
                        'description': 'Click-through rate (CTR)',
                        'type': 'array'
                    },
                    'bounce_rate': {
                        'description': 'Bounce rate percentage',
                        'type': 'array'
                    },
                    'targets': {
                        'description': 'Target impressions and clicks',
                        'type': 'array'
                    },
                    'pricing_model': {
                        'description': 'Pricing model used (CPM, CPC, etc.)',
                        'type': 'array'
                    }
                }
            },
            'budget': {
                'description': 'Budget information',
                'type': 'object',
                'properties': {
                    'daily_budget': {
                        'description': 'Daily budget amount',
                        'type': 'array'
                    },
                    'total_budget': {
                        'description': 'Total campaign budget',
                        'type': 'array'
                    }
                }
            },
            'targeting': {
                'description': 'Campaign targeting parameters',
                'type': 'object',
                'properties': {
                    'geography': {
                        'description': 'Geographic targeting locations',
                        'type': 'array'
                    },
                    'demographics': {
                        'description': 'Demographic targeting criteria',
                        'type': 'array'
                    },
                    'age_range': {
                        'description': 'Age range targeting',
                        'type': 'array'
                    },
                    'devices': {
                        'description': 'Device targeting',
                        'type': 'array'
                    }
                }
            }
        }
    }
}
"""

In [ ]:
# Alternative response format: Array-of-arrays (simpler than JSON schema)
# Each inner array contains [label, question]
# This format returns a flat structure instead of nested objects
alt_resp_schema = [
    # Vendor information
    ['vendor_name', 'What is the name of the vendor or advertising service provider?'],
    
    # Transaction details
    ['receipt_id', 'What is the receipt or invoice ID number?'],
    ['transaction_date', 'What is the date of the transaction?'],
    ['payment_method', 'What payment method was used?'],
    
    # Customer information
    ['customer_name', 'What is the name of the customer contact person?'],
    ['company_name', 'What is the name of the customer company or organization?'],
    
    # Campaign details
    ['campaign_name', 'What is the name of the advertising campaign?'],
    ['content_types', 'What types of content are included (Display, Video, etc.)?'],
    ['ad_formats', 'What ad formats are used in the campaign?'],
    ['period_startdate', 'What is the campaign period start date?'],
    ['period_enddate', 'What is the campaign period end date?'],
    
    # Financial details
    ['line_items', 'List all the individual line items on the receipt'],
    ['subtotal', 'What is the subtotal amount before tax?'],
    ['tax', 'What is the tax amount?'],
    ['total', 'What is the total amount charged?'],
    
    # Performance metrics
    ['cpm', 'What is the cost per thousand impressions (CPM)?'],
    ['ctr', 'What is the click-through rate (CTR)?'],
    ['bounce_rate', 'What is the bounce rate percentage?'],
    ['targets', 'What are the target impressions and clicks?'],
    ['pricing_model', 'What pricing model is used (CPM, CPC, CPA, etc.)?'],
    
    # Budget information
    ['daily_budget', 'What is the daily budget amount?'],
    ['total_budget', 'What is the total campaign budget?'],
    
    # Targeting parameters
    ['geography', 'What geographic locations are being targeted?'],
    ['demographics', 'What demographic criteria are being targeted?'],
    ['age_range', 'What age range is being targeted?'],
    ['devices', 'What devices are being targeted?']
]

print(f"Alternative schema defines {len(alt_resp_schema)} extraction questions")


### Comparing Response Formats

**JSON Schema (`resp_schema`):**
- ✅ **Nested Structure**: Returns hierarchical data (`vendor.vendor_name`, `transaction.date`)
- ✅ **Type Definitions**: Specifies that all fields are arrays
- ✅ **Organized**: Groups related fields into objects
- ⚠️ **Complex**: Requires understanding JSON schema syntax
- 📊 **Access**: `extracted_data:response.vendor.vendor_name[0]`

**Array-of-Arrays (`alt_resp_schema`):**
- ✅ **Simple Format**: Easy to understand and modify
- ✅ **Natural Language**: Each field has a clear question
- ✅ **Flat Structure**: Returns all fields at top level
- ⚠️ **No Nesting**: Loses organizational hierarchy
- 📊 **Access**: `extracted_data:response.vendor_name` (no nesting)

**Note**: The current notebook uses `resp_schema` (nested JSON). To use `alt_resp_schema`, replace `{resp_schema}` with `{alt_resp_schema}` in the build_extraction_query cell and update the flattening queries to remove nested paths.


## Step 6: Extract Structured Data with AI_EXTRACT (Direct from PDFs)

Using Snowflake's AI_EXTRACT to process PDF files directly from the stage - no parsing step required!

### Incremental Extraction:
- **CREATE TABLE IF NOT EXISTS**: Preserves existing extracted data
- **INSERT INTO**: Adds only new extractions
- **WHERE NOT IN**: Only processes receipts not already in extracted_receipt_data_via_ai_extract
- **Saves Costs**: Avoids re-running expensive AI_EXTRACT on same receipts

### AI_EXTRACT Direct File Processing:
- ✅ **Direct PDF Access**: Uses `TO_FILE` to read PDFs from stage
- ✅ **Single Step**: No need for separate AI_PARSE_DOCUMENT step
- ✅ **Structured Schema**: Define exact JSON schema for consistent results
- ✅ **Nested Objects**: Support complex hierarchical structures (vendor, transaction, campaign, etc.)

The AI reads PDF files directly from the stage and extracts data according to the defined schema, ensuring vendor details, transaction info, campaign details, financial totals, performance metrics, and targeting parameters are properly structured.

In [ ]:
# Create extracted_receipt_data_via_ai_extract table if it doesn't exist
session.sql("""
CREATE TABLE IF NOT EXISTS 
extracted_receipt_data_via_ai_extract (
    relative_path STRING,
    extracted_data VARIANT
)
""").collect()

# Extract directly from PDF files on the stage using AI_EXTRACT
# AI_EXTRACT can work with files directly without needing AI_PARSE_DOCUMENT first
query = f"""
INSERT INTO extracted_receipt_data_via_ai_extract
SELECT
    relative_path,
    AI_EXTRACT(
        file => TO_FILE('@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS', relative_path),
        responseFormat => {resp_schema}
    ) as extracted_data
FROM DIRECTORY(@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS)
WHERE relative_path NOT IN (SELECT relative_path FROM extracted_receipt_data_via_ai_extract)
"""

print("Query prepared for AI_EXTRACT processing directly from PDF files")


In [ ]:
-- Just uncomment in case you want all new receipts extraction:

--truncate table extracted_receipt_data_via_ai_extract;
--select * from extracted_receipt_data_via_ai_extract;

In [ ]:
result = session.sql(query).collect()
# Get the actual number of rows inserted from the result metadata
rows_inserted = result[0]['number of rows inserted'] if result else 0
print(f"✓ Extracted data from {rows_inserted} new receipt(s) using AI_EXTRACT")


## Step 7: Preview Extracted Receipt Data

Each row contains a complete structured representation of a receipt with all extracted fields in VARIANT format, ready for flattening and analysis.


In [ ]:
# View the extracted data
result_df = session.table('extracted_receipt_data_via_ai_extract').to_pandas()
result_df.head(100)


## Summary

### What We've Accomplished:

1. ✅ **Direct PDF Extraction**: Used AI_EXTRACT to process PDFs directly from stage
2. ✅ **Structured Schema**: Defined responseFormat with nested objects
3. ✅ **Stored Data**: Populated `extracted_receipt_data_via_ai_extract` table with structured information

### Tables Created:
1. `extracted_receipt_data_via_ai_extract` - Structured data extracted with AI_EXTRACT

### For Analytics:
Run `receipts-analysis/analysis.sql` which will:
- Create `receipt_analytics_ai_extract_vw` (flattened view)
- Provide ready-to-use analytical queries

---

**Your receipt data is now extracted with AI_EXTRACT and ready for analytics!** 📊
